# 写在前面

这个文件基本上是参考了 <a href="https://cosx.org/2013/01/lda-math-mcmc-and-gibbs-sampling">LDA-math-MCMC 和 Gibbs Sampling (靳志辉
)</a> 这篇文章，只是减掉了很多东西，方便翻阅。

# Markov chain

$$P(X_{t+1}=x|X_t,X_{t-1},...) = P(X_{t+1}=x|X_t)$$ 状态转移的概率只依赖于前一个状态。

In [1]:
import numpy as np

P = [
    [0.65, 0.28, 0.07],
    [0.15, 0.67, 0.18],
    [0.12, 0.36, 0.52]
]

p1 = [0.21, 0.68, 0.11]
p2 = [0.75, 0.15, 0.1 ]
p3 = [0,    1,    0   ]

np.set_printoptions(precision=3)

print(0, ': ', p1, '\t', p2, '\t', p3)

for i in range(9) :
    p1 = np.matmul(p1, P)
    p2 = np.matmul(p2, P)
    p3 = np.matmul(p3, P)
    print(i+1,': ',p1, '\t', p2, '\t', p3)

0 :  [0.21, 0.68, 0.11] 	 [0.75, 0.15, 0.1] 	 [0, 1, 0]
1 :  [0.252 0.554 0.194] 	 [0.522 0.346 0.132] 	 [0.15 0.67 0.18]
2 :  [0.27  0.512 0.218] 	 [0.407 0.426 0.167] 	 [0.22  0.556 0.225]
3 :  [0.278 0.497 0.225] 	 [0.349 0.459 0.192] 	 [0.253 0.515 0.232]
4 :  [0.282 0.492 0.226] 	 [0.318 0.475 0.207] 	 [0.27  0.499 0.231]
5 :  [0.284 0.49  0.226] 	 [0.303 0.482 0.215] 	 [0.278 0.493 0.229]
6 :  [0.285 0.489 0.225] 	 [0.295 0.485 0.22 ] 	 [0.282 0.491 0.227]
7 :  [0.286 0.489 0.225] 	 [0.291 0.487 0.222] 	 [0.284 0.49  0.226]
8 :  [0.286 0.489 0.225] 	 [0.289 0.488 0.224] 	 [0.285 0.489 0.226]
9 :  [0.286 0.489 0.225] 	 [0.288 0.488 0.224] 	 [0.286 0.489 0.225]


可以看到，初状态为 [0.21, 0.68, 0.11] 时，第7次迭代开始，状态已经稳定。其他初始状态也能够在一定迭代次数后收敛。说明这个收敛与初状态无关，是状态转移矩阵$P$的性质

In [2]:
P_origin = P
for i in range(20) :
    P = np.matmul(P, P_origin)
pi = P[0]
print(P, '\n')
print(np.matmul(P, P_origin), '\n')
print(pi, '\n', np.matmul(pi, P_origin))

[[0.287 0.489 0.225]
 [0.287 0.489 0.225]
 [0.287 0.489 0.225]] 

[[0.287 0.489 0.225]
 [0.287 0.489 0.225]
 [0.287 0.489 0.225]] 

[0.287 0.489 0.225] 
 [0.287 0.489 0.225]


上面代码块表明 $P^a = P^b  (a,b \geq 20)$ 。

当 $n$ 足够大时，$P^n$ 的每一行都会收敛到同一个分布 $\pi^T = [\pi_1, \pi_2, ...] (\pi_i \in R_+)$ 。$$P^n = [\pi, \pi, ..., \pi]^T$$ 收敛现象是大多数马尔科夫链的性质。方便起见，后面都把 $\pi$ 当作 row vector 而非 column vector 。$\pi$ 称为马尔科夫链的**平稳分布**，是方程 $\pi P = \pi$ 的唯一非负解。

# Markov Chain Monte Carlo

假设有某个概率分布 $p(x)$ ，直接从 $p(x)$ 抽样是困难的，想要抽出服从 $p(x)$ 的样本只能用别的方法生成样本。

前面说了马尔科夫链能收敛到一个平稳分布，如果能构造一个转移矩阵为 $P$ 的马尔科夫链，使得马尔科夫链的平稳分布恰好是 $p(x)$ ，那么我们从随机初始状态 $x_0$ 开始不断跳转得到一个序列，前面若干步跳转是未收敛的，属于 burn-in 阶段，就像开头社会阶层的例子一样，前面几次迭代没有收敛。足够多的跳转次数之后，假设在第$n$步收敛，那么后面的所有序列 $x_n, x_{n+1}, x_{n+2}, ...$ 的样本分布就收敛到稳定分布，服从 $p(x)$ ，此时我们就成功从难采样的概率分布 $p(x)$ 采到服从分布的样本了。

**Theorem: [细致平稳条件]** 一个非周期的马尔科夫链的转移矩阵 $P$ 与任意一个分布 $\pi$ ，如果 $$\pi_i P_{ij} = \pi_j P_{ji}$$ 那么 $\pi$ 就是这个马尔科夫链的平稳分布。

现在假设我们已经有一个转移矩阵为 $Q$ 的马尔科夫链，通常情况下 $p(i)Q_{ij} \neq p(j)Q_{ji}$ ，因为很难恰好选中一个矩阵 $Q$ 使等号成立。引入 $$\alpha(i,j)=p(j)Q_{ji} \quad , \quad \alpha(j,i)=p(i)Q_{ij}$$ 得到等式 $$p(i)Q_{ij}\alpha(i,j) = p(j)Q_{ji}\alpha(j,i)$$ 替换一下符号得到 $$p(i)Q'_{ij} = p(j)Q'_{ji}$$ 此时我们得到一个转移矩阵为 $Q'$ 的马尔科夫链， $p(x)$ 与 $Q'$ 满足细致平稳条件，因此 $p(x)$ 是 $Q'$ 的平稳分布。

引入的 $\alpha(i,j)$ 称为接受率，可理解为在原马尔科夫链上做 $i \rightarrow j$ 跳转时我们以 $\alpha(i,j)$ 的概率接受这次跳转。于是得到新马尔科夫链 $Q'$ 的转移概率为 $Q'_{ij} = Q_{ij}\alpha(i,j)$

现在的问题是，如果 $\alpha(i,j)$ 太小，就会总是拒绝跳转，马尔科夫链就长期原地踏步，遍历样本空间就需要花费大量时间。为了提高收敛速度，需要提高接受率。当把 $\alpha(i,j)$ 和 $\alpha(j,i)$ 同时扩大相同倍数，细致平稳条件的等式仍成立，但接受率仍要不大于 1 ，因此我们把所有的 $\alpha(i,j)$ 扩大为 $$\alpha(i,j) = min \bigg\{ \frac{p(j)Q_{ji}}{p(i)Q_{ij}} \quad , \quad 1 \bigg\}$$ 这个改造就是 Metropolis-Hastings 算法。

# Gibbs Sampling

接受率 $\alpha(i,j) \leq 1$ 的存在使得在高维情况下 Metropolis-Hastings 算法的效率也不高。想要找到一个接受率为 1 的转移矩阵。

同时，我们考察一种情况: 高维空间的分布 $p(X)$ 直接计算非常困难，但条件概率分布 $p\bigg(x_i\bigg|\big\{ X$ \ $x_i \big\}\bigg)$ 很容易。

先考察二维空间。假设 $A(x_1,y_1)$ ， $B(x_1,y_2)$ ， $C(x_2,y_1)$ 有 $$p(x_1,y_1)p(y_2|x_1)=p(x_1)p(y_1|x_1)p(y_2|x_1)$$ $$p(x_1,y_2)p(y_1|x_1)=p(x_1)p(y_2|x_1)p(y_1|x_1)$$ 因此 $$p(x_1,y_1)p(y_2|x_1)=p(x_1,y_2)p(y_1|x_1)$$ 即 $$p(A)p(y_2|x_1)=p(B)p(y_1|x_1)$$ 即仅在Y维度上的状态转移满足细致平稳条件。同理可得 $$p(A)p(x_2|y_1)=p(C)p(x_1|y_1)$$ 即仅在X维度上的状态转移满足细致平稳条件。

所以在二维空间里，在任意一维上的状态转移满足细致平稳条件。所以构造一个状态转移矩阵 $Q$ ，使得当且仅当任意两个状态之间仅有一个维度不同时，状态转移矩阵对应的 entry 不为零
$$Q(A \rightarrow B) = p(y_B|x_1) \quad , \quad if \quad x_A=x_B=x_1$$
$$Q(A \rightarrow C) = p(x_C|y_1) \quad , \quad if \quad y_A=y_C=y_1$$
$$Q(A \rightarrow D) = 0 \quad , \quad otherwise$$
可验证任意两个状态 $p(J)Q(J \rightarrow K)=p(K)Q(K \rightarrow J)$ 。即任意两个状态 $J$ 和 $K$ 之间的跳转满足细致平稳条件，所以我们构造的这个马尔科夫链 $Q$ 最后能够收敛到平稳分布 $p(X)$

以上内容可以推广到更高维空间。所以如果每次跳转都只沿着一个坐标维度跳，那么每次跳转都是满足细致平稳条件，那么跳转次数足够多之后就会收敛到平稳分布 (有点类似于坐标下降) 。收敛前的跳转得到的样本不服从 $p(X)$ ，此阶段称为 burn-in，收敛后继续跳转的得到的后续样本服从 $p(X)$。注意，收敛后取得的样本并不相互独立，但我们需要的只是让样本服从 $p(X)$ ，没有要求独立。

因此想要从一个高维空间的概率分布 $p(X)$ 采样， Gibbs sampling 的算法就是随机生成一个初始样本，然后每次都只选择一个维度 $x_i$ (随机或按顺序) 进行跳转，跳转的距离就由 $p\bigg(x_i\bigg|\big\{ X$ \ $x_i \big\}\bigg)$ 决定。

在以上推导过程中，没有出现接受率 $\alpha$ ，沿着任一坐标维度跳转的接受率都是 1 。如果不沿着单一坐标维度跳转，状态转移矩阵对应的 entry 根据定义为 0 ，根本不会出现这种跳转。因此每次跳转都被接受。

Reference:
1. https://cosx.org/2013/01/lda-math-mcmc-and-gibbs-sampling